In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
from one.api import ONE
import matplotlib.pyplot as plt

from utils.load_meso_session import MesoscopeSession

one = ONE()

# Load your session
session = MesoscopeSession.from_csv(one, '../good_mesoscope_sessions_final.csv', 0)

# Get the activity matrix as numpy arrays
activity_matrix, timestamps = session.get_activity_matrix()
print(f"Activity matrix shape: {activity_matrix.shape}")  # [time_points, neurons]
print(f"Timestamps shape: {timestamps.shape}")

ModuleNotFoundError: No module named 'utils'

In [ ]:
print(f"Activity matrix shape: {activity_matrix.shape}")  # [time_points, neurons]

# Create training data: predict time t+1 from time t
X = activity_matrix[:-1, :]  # All neurons at time t (timestamps 0 to n-2)
y = activity_matrix[1:, :]   # All neurons at time t+1 (timestamps 1 to n-1)

# Use first 1000 time points for training
n_train = min(10000, X.shape[0])
X_train = X[:n_train, :]     # [1000, n_neurons]
y_train = y[:n_train, :]     # [1000, n_neurons]

# Test on timestamp 1001 (predict from timestamp 1000)
X_test = X[n_train:n_train+1, :]  # [1, n_neurons] - activity at timestamp 1000
y_test = y[n_train:n_train+1, :]  # [1, n_neurons] - actual activity at timestamp 1001

print(f"Training data: {X_train.shape} -> {y_train.shape}")
print(f"Test data: {X_test.shape} -> {y_test.shape}")
print(f"Predicting timestamp {n_train+1} from timestamp {n_train}")

# Fit ONE linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict all neurons at timestamp 1001
y_pred = model.predict(X_test)  # [1, n_neurons]

# Calculate metrics
r2 = r2_score(y_test.flatten(), y_pred.flatten())
mse = mean_squared_error(y_test.flatten(), y_pred.flatten())

print(f"\nResults:")
print(f"R² score: {r2:.4f}")
print(f"MSE: {mse:.6f}")
print(f"Predicted {y_pred.shape[1]} neurons simultaneously")

Activity matrix shape: (19081, 7673)
Training data: (10000, 7673) -> (10000, 7673)
Test data: (1, 7673) -> (1, 7673)
Predicting timestamp 10001 from timestamp 10000

Results:
R² score: -2.2382
MSE: 37551.710938
Predicted 7673 neurons simultaneously


In [ ]:
y_pred

NameError: name 'y_pred' is not defined

In [ ]:
# Just predict a manageable number of neurons (e.g., 50-100)
n_target_neurons = 50
target_neurons = range(n_target_neurons)

X_train = activity_matrix[:1000, :]  # All neurons as features
y_train = activity_matrix[1:1001, target_neurons]  # Only predict subset

X_test = activity_matrix[1000:1001, :]  # All neurons at timestamp 1000
y_test = activity_matrix[1001:1002, target_neurons]  # Subset at timestamp 1001

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test.flatten(), y_pred.flatten())
print(f"Predicting {n_target_neurons} neurons: R² = {r2:.4f}")

Predicting 50 neurons: R² = -19.2017


In [ ]:
# Pick one interesting neuron and do it properly
target_neuron = 0

X_train = activity_matrix[:1000, :]  # All neurons as features
y_train = activity_matrix[1:1001, target_neuron]  # Single neuron target

X_test = activity_matrix[1000:1001, :]
y_test = activity_matrix[1001:1002, target_neuron]

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(f"Predicting neuron {target_neuron}: Actual = {y_test[0]:.4f}, Predicted = {y_pred[0]:.4f}")

Predicting neuron 0: Actual = 0.0000, Predicted = 139.7676
